# Testing the API

Satellite data starts being recored on {'date': '2015-06-13'}

In [1]:
import requests
import os
from IPython.display import Image

In [2]:
# All Data
url = 'https://epic.gsfc.nasa.gov/api/natural/all'

# Most Recent
url = 'https://epic.gsfc.nasa.gov/api/natural/'

# URL by date
date = '20180428'
url = f'https://epic.gsfc.nasa.gov/api/enhanced/date/{date}'
res = requests.get(url)

In [3]:
# Example data retrieved
print(f'There are {len(res.json())} images available')
res.json()[0]

There are 22 images available


{'identifier': '20180428005516',
 'caption': "This image was taken by NASA's EPIC camera onboard the NOAA DSCOVR spacecraft",
 'image': 'epic_RGB_20180428005516',
 'version': '02',
 'centroid_coordinates': {'lat': 21.489258, 'lon': 165.871582},
 'dscovr_j2000_position': {'x': 1082471.700001,
  'y': 765002.975361,
  'z': 519622.982812},
 'lunar_j2000_position': {'x': -371472.817493,
  'y': -85970.908502,
  'z': 29631.801617},
 'sun_j2000_position': {'x': 119549193.299457,
  'y': 84022262.300594,
  'z': 36423693.849281},
 'attitude_quaternions': {'q0': 0.1799,
  'q1': 0.04705,
  'q2': -0.936977,
  'q3': 0.295793},
 'date': '2018-04-28 00:50:27',
 'coords': {'centroid_coordinates': {'lat': 21.489258, 'lon': 165.871582},
  'dscovr_j2000_position': {'x': 1082471.700001,
   'y': 765002.975361,
   'z': 519622.982812},
  'lunar_j2000_position': {'x': -371472.817493,
   'y': -85970.908502,
   'z': 29631.801617},
  'sun_j2000_position': {'x': 119549193.299457,
   'y': 84022262.300594,
   'z': 36

In [4]:
# Available data time and coords
fetch = 3
points = []
for e in res.json()[:fetch]:
    points.append((e['image'], e['centroid_coordinates'], e['date']))
points

[('epic_RGB_20180428005516',
  {'lat': 21.489258, 'lon': 165.871582},
  '2018-04-28 00:50:27'),
 ('epic_RGB_20180428020044',
  {'lat': 21.489258, 'lon': 149.458008},
  '2018-04-28 01:55:55'),
 ('epic_RGB_20180428030612',
  {'lat': 21.533203, 'lon': 133.132324},
  '2018-04-28 03:01:23')]

In [5]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/{size['med']}/{res.json()[0]['image']}.{size['med']}"
print(img)
display(Image(url=img))

https://epic.gsfc.nasa.gov/archive/enhanced/2018/04/28/jpg/epic_RGB_20180428005516.jpg


In [6]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
for e in res.json():
    img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/png/{e['image']}.{size['big']}"
    #display(Image(url=img))

In [7]:
# Example data retrieved
print(res.json()[0]['identifier'])
YYYYMMDD = res.json()[0]['identifier']
year = YYYYMMDD[:4]
month = YYYYMMDD[4:6]
day = YYYYMMDD[6:8]
print(year, month, day)

20180428005516
2018 04 28


# Basic getSat() by date

In [8]:
## VERSION 1
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{YYYY}-{MM}-{DD}"
    res = requests.get(metadata, params=queryParams)
    print(res.status_code, res.url)

    # Construct the resource url
    img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"

    # Return json from requests' response body
    return res.json()

#getSat('nat', '2020', '04', '09')[1]

In [9]:
## VERSION 02
# Define when
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    date = f"{YYYY}-{MM}-{DD}"
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    # Construct the resource url
    #img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"
    
    #Return relevant data as list
    return data

#getSat('nat', '2020', '03', '10')

In [10]:
## VERSION 03
# Define when
def getSat(collection, img_type,YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the resource url
    if len(data) > 0:
        for e in data:
            print(e)
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            print(img)
    #Return relevant data as list
    return data

#getSat('nat', 'thumb', '2020', '03', '10')

In [11]:
## VERSION 04
def getSat(collection, img_type, YYYY,MM,DD, queryParams=dict()):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the image resource url
    satellite_images = []
    if len(data) > 0:
        for e in data:
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            satellite_images.append([img, e[1]])
    #Return relevant data as list
    return satellite_images

images = getSat('enh', 'thumb', '2020', '03', '01')

200 https://epic.gsfc.nasa.gov/api/enhanced/date/2020-03-01
There are 12 satellite images available for this date: 2020-03-01


In [12]:
display(Image(url=images[0][0]))

# Working with cleaned dataset dates

In [13]:
import pandas as pd
import numpy as np
import datetime

In [22]:
df = pd.read_csv('OUTPUT/volcanic-explosions.csv')
df = df.drop(columns='Unnamed: 0')

In [27]:
# Select an event on the dataframe for which we know there is available data in the API
event_date = df.iloc[20]['start']
event_date = event_date.split('-')
print(event_date)

# GET the API image and coordinates
available_data = getSat('nat', 'png', *event_date)
print(available_data[0])
print(available_data[-1])

['2019', '06', '22']
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-06-22
There are 22 satellite images available for this date: 2019-06-22
['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/png/epic_1b_20190622004555.png', {'lat': 20.357666, 'lon': 173.430176}]
['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/png/epic_1b_20190622234040.png', {'lat': 20.126953, 'lon': -170.551758}]


# Finding the satellite images for the START of each eruptive phase

In [35]:
# Access the START dates on each eruptive phase
for phase in df['start'][5:20]:
    print(phase)
    print(getSat('nat', 'thumb', *phase.split('-')))

2019-12-09
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-09
There are 0 satellite images available for this date: 2019-12-09
[]
2019-12-07
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-07
There are 0 satellite images available for this date: 2019-12-07
[]
2019-12-05
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-05
There are 0 satellite images available for this date: 2019-12-05
[]
2019-11-07
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-11-07
There are 0 satellite images available for this date: 2019-11-07
[]
2019-11-02
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-11-02
There are 0 satellite images available for this date: 2019-11-02
[]
2019-10-24
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-10-24
There are 0 satellite images available for this date: 2019-10-24
[]
2019-10-13
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-10-13
There are 0 satellite images available for this date: 2019-10-13
[]
2019-08-07
200 https://epic.gsfc.n

In [53]:
# Access the START dates on each eruptive phase
for phase in df['start'][::15]:
    print(phase)
    print(getSat('nat', 'thumb', *phase.split('-')))

2020-02-10
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-02-10
There are 0 satellite images available for this date: 2020-02-10
[]
2019-07-23
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-07-23
There are 0 satellite images available for this date: 2019-07-23
[]
2019-02-18
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-02-18
There are 13 satellite images available for this date: 2019-02-18
[['https://epic.gsfc.nasa.gov/archive/natural/2019/02/18/thumbs/epic_1b_20190218001751.jpg', {'lat': -19.533691, 'lon': 171.254883}], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/18/thumbs/epic_1b_20190218020554.jpg', {'lat': -19.504395, 'lon': 144.177246}], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/18/thumbs/epic_1b_20190218035357.jpg', {'lat': -19.434814, 'lon': 117.224121}], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/18/thumbs/epic_1b_20190218054200.jpg', {'lat': -19.423828, 'lon': 90.219727}], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/18/

200 https://epic.gsfc.nasa.gov/api/natural/date/2017-01-23
There are 12 satellite images available for this date: 2017-01-23
[['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_20170123001751.jpg', {'lat': -24.484863, 'lon': -174.338379}], ['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_20170123020554.jpg', {'lat': -24.470215, 'lon': 158.679199}], ['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_20170123035357.jpg', {'lat': -24.455566, 'lon': 131.638184}], ['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_20170123054200.jpg', {'lat': -24.440918, 'lon': 104.597168}], ['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_20170123073003.jpg', {'lat': -24.411621, 'lon': 77.607422}], ['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_20170123091806.jpg', {'lat': -24.382324, 'lon': 50.57373}], ['https://epic.gsfc.nasa.gov/archive/natural/2017/01/23/thumbs/epic_1b_2017012

In [24]:
0/0

ZeroDivisionError: division by zero

In [16]:
date_rng = pd.date_range(start='1/1/2015', end='10/04/2020', freq='D')
date_rng

DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2020-09-25', '2020-09-26', '2020-09-27', '2020-09-28',
               '2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02',
               '2020-10-03', '2020-10-04'],
              dtype='datetime64[ns]', length=2104, freq='D')

In [17]:
df = pd.DataFrame(date_rng, columns=['date'])
df['data'] = np.random.randint(0,100, size=(len(date_rng)))
df.head()

,date,data
0,2015-01-01,91
1,2015-01-02,61
2,2015-01-03,32
3,2015-01-04,90
4,2015-01-05,25


In [18]:
df['datetime'] = pd.to_datetime(df.date)
df = df.set_index('datetime')

#list only the second day of each month
#df[df.index.day == 2]

# calculate mean of each year
df.resample('M').mean()

,data
datetime,
2015-01-31,48.741935
2015-02-28,43.464286
2015-03-31,45.322581
2015-04-30,47.533333
2015-05-31,56.000000
...,...
2020-06-30,56.300000
2020-07-31,49.548387
2020-08-31,54.129032


In [19]:
year= 2020
month= 2
day=6

datetime.date(year,month,day).weekday()

3

In [20]:
datetime.date.today()

datetime.date(2020, 4, 11)